In [0]:
import tensorflow as tf
import numpy as np

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = np.array([[0, 1, 0, 2, 3, 3]])   # hihell
x_one_hot = np.array([[[1, 0, 0, 0, 0],   # h 0
                      [0, 1, 0, 0, 0],   # i 1
                      [1, 0, 0, 0, 0],   # h 0
                      [0, 0, 1, 0, 0],   # e 2
                      [0, 0, 0, 1, 0],   # l 3
                      [0, 0, 0, 1, 0]]]) # l 3

y_data = np.array([[1, 0, 2, 3, 3, 4]])    # ihello

print(x_one_hot.shape)
print(y_data.shape)

(1, 6, 5)
(1, 6)


In [0]:
num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

In [0]:
X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

print(outputs.shape)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
print(outputs.shape)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
(1, 6, 5)
(1, 6, 5)


In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.6607363 prediction:  [[3 3 3 3 3 3]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  llllll
1 loss: 1.544876 prediction:  [[3 3 3 3 3 3]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  llllll
2 loss: 1.4733824 prediction:  [[3 3 3 3 3 3]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  llllll
3 loss: 1.3948791 prediction:  [[3 3 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lllllo
4 loss: 1.3064575 prediction:  [[3 3 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lllllo
5 loss: 1.2172747 prediction:  [[3 3 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lllllo
6 loss: 1.1180015 prediction:  [[3 3 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lllllo
7 loss: 1.0127679 prediction:  [[3 3 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lllllo
8 loss: 0.912996 prediction:  [[3 0 3 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  lhlllo
9 loss: 0.817923 prediction:  [[0 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
	Prediction str:  hhello
10 loss: 0.7372184 prediction:  [

In [0]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size, sequence_length]
weight = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weight)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print("Loss: ", sequence_loss.eval())

Loss:  0.5967595
